In [2]:
import pandas as pd

from EDA_tools import DfAnalysis, DataPreprocessing
from DimRed_plus import PCAAnalysis

from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score 

import os

import warnings
warnings.filterwarnings('ignore')

In [3]:
agence = pd.read_csv(r"data\agence.csv")
france = pd.read_csv(r"data\france.csv")
insee_commune = pd.read_csv(r"data\insee_commune.csv")
localisation_commune = pd.read_csv(r"data\localisation_commune.csv")
map_code_insee = pd.read_csv(r"data\map_code_insee.csv")

## Construction du dataset d'étude

Nous disposons de plusieurs datasets (agence, france, insee_commune, localisation), il serait intéressant de les combiner de telle sorte à pouvoir en extraire les données essentielles lors d'une EDA plus approfondie mais surtout afin de le fournir à nos modèles de machine learning que nous implémenterons dans un second jupyter notebook.

Nous utiliserons notamment deux fichiers `DimRed_plus` et `EDA_tools` qui proposent certaines fonctionnalités d'automatisation de graphiques, d'étapes de preprocessing des données etc...

## Sommaire

Au cours de ce notebook Jupyter, nous entreprendrons les étapes suivantes pour enrichir notre jeu de données initial (`agence.csv`) avec des informations provenant d'autres fichiers afin de mieux correspondre au profil client et d'étudier l'impact de la performance des agents immobiliers sur la détermination de la valeur foncière des biens immobiliers en Haute-Garonne :

1. **Analyse rapide du fichier agence.csv** :
   - Exploration initiale des données pour comprendre leur structure et leur contenu.

2. **Fusion avec le fichier france.csv** :
   - Intégration des données sur la France pour enrichir notre jeu de données.

3. **Analyse du fichier insee_commune.csv** :
   - Exploration des données de l'INSEE sur les communes pour obtenir des informations supplémentaires.

4. **Fusion entre les fichiers insee_commune.csv et localisation_commune.csv** :
   - Combinaison des données de localisation des communes avec les données de l'INSEE pour une meilleure compréhension géographique.

5. **Analyse des corrélations entre les variables** :
   - Identification des relations et des dépendances entre les différentes caractéristiques du jeu de données.

6. **Sélection manuelle des variables** :
   - Choix des variables pertinentes à inclure dans notre modèle.

7. **Préparation d'un jeu de données pour le machine learning** :
   - Transformation des données en un format adapté à l'apprentissage automatique.

8. **Test avec un premier modèle CatBoost** :
   - Mise en œuvre d'un modèle de prédiction basé sur l'algorithme CatBoost pour évaluer sa performance dans la prédiction de la valeur foncière des biens immobiliers.

À travers ces étapes, nous visons à améliorer la qualité de notre jeu de données, à identifier les facteurs clés influençant la valeur foncière et à construire un modèle prédictif robuste pour soutenir nos analyses futures.

## Etapes 1, 2

In [4]:
agence.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date_mutation              4000 non-null   object 
 1   valeur_fonciere            4000 non-null   float64
 2   code_postal                4000 non-null   int64  
 3   nom_commune                4000 non-null   object 
 4   id_parcelle                4000 non-null   object 
 5   type_local                 4000 non-null   object 
 6   surface_reelle_bati        4000 non-null   float64
 7   surface_terrain            4000 non-null   float64
 8   nombre_pieces_principales  4000 non-null   float64
 9   agent                      4000 non-null   int64  
 10  prixcible                  4000 non-null   int64  
 11  prixvente_initial          4000 non-null   int64  
 12  evaluation_agent           4000 non-null   object 
dtypes: float64(4), int64(4), object(5)
memory usage:

Dans un premier temps:
- Nous effectuons la conversion de type des dates 
- Nous vérifions les valeurs manquantes dans le dataset en créant une instance de `DfAnalysis`
    - Nous ajouterons le préfixe 'dan' pour différencier cette instance du dataset original

Note: l'étude complète (outliers, corrélations...) du dataset interviendra plus tard

In [5]:
agence['date_mutation'] = pd.to_datetime(agence['date_mutation'], format='%Y-%m-%d')

In [6]:
dan_agence = DfAnalysis(agence)
dan_agence.missing_values_check()
print(50*'-')
dan_agence.duplicate_check()

4000 row(s) have 0 missing values
Total number of rows with missing values: 0
List of indexes of rows with missing values: []
--------------------------------------------------
No duplicated rows


Dans un second temps: 
- Nous ne sélectionnons que la plage de données de `france.csv` concernant la haute garonne

`france.csv` est un dataset intéressant en ce qu'il réunit des informations relatives au carreau d'appartenance des biens considérés (un carreau est une unité de 200m de côté sur laquelle peuvent être considérées des variables comme la densité de population, le niveau de vie des ménages etc...): ceci en fait un dataset très fin et dons utile pour notre étude

In [7]:
france['code_postal'] = france['code_postal'].astype(str)
france_HG = france[france['code_postal'].str.startswith('31')]
france_HG.sample(2)

,date_mutation,valeur_fonciere,adresse_code_voie,code_postal,nom_commune,code_commune,id_parcelle,type_local,surface_reelle_bati,surface_terrain,...,carreau_Ind_11_17,carreau_Ind_18_24,carreau_Ind_25_39,carreau_Ind_40_54,carreau_Ind_55_64,carreau_Ind_65_79,carreau_Ind_80p,carreau_Ind_inc,carreau_I_pauv,carreau_t_maille
621964,2015-08-18,84000.0,3044,31200,Toulouse,31555,31555830AM0276,Appartement,50.0,50.0,...,271.9,322.0,1562.0,867.0,348.1,301.0,112.1,77.0,0,1000
662173,2017-12-14,206500.0,0018,31470,Fontenilles,31188,311880000B0710,Maison,83.0,1770.0,...,124.6,54.6,114.0,244.2,79.0,64.9,17.0,31.0,0,1000


In [8]:
print('france_HG',france_HG.shape, '|', 'agence', agence.shape)

france_HG (57661, 46) | agence (4000, 13)


Nous effectuons quelques conversions de type

In [9]:
france_HG['date_mutation'] = pd.to_datetime(france_HG['date_mutation'], format='%Y-%m-%d')

On constate ici que les coordonnées géographiques de `france.csv` concernent les biens eux-mêmes cela peut nous donner des idées de combinaisons de features interessantes pour l'avenir:
- Distance du bien au centre ville de sa commune d'appartenance en utilisant `localisation_commune.csv`
- Distance du bien au centre ville de Toulouse

In [10]:
france_HG[['nom_commune', 'latitude', 'longitude']].head()

,nom_commune,latitude,longitude
42517,Bourbon-l'Archambault,46.600380,3.070512
42518,Bourbon-l'Archambault,46.597794,3.080006
42918,Bourbon-l'Archambault,46.586081,3.060649
42919,Bourbon-l'Archambault,46.585826,3.066112
42920,Bourbon-l'Archambault,46.579173,3.055870


In [11]:
france.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2641120 entries, 0 to 2641119
Data columns (total 46 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   date_mutation              object 
 1   valeur_fonciere            float64
 2   adresse_code_voie          object 
 3   code_postal                object 
 4   nom_commune                object 
 5   code_commune               object 
 6   id_parcelle                object 
 7   type_local                 object 
 8   surface_reelle_bati        float64
 9   surface_terrain            float64
 10  nombre_pieces_principales  float64
 11  longitude                  float64
 12  latitude                   float64
 13  departement                int64  
 14  index_right                int64  
 15  carreau_Id_carr_n          object 
 16  carreau_Ind                float64
 17  carreau_Men                float64
 18  carreau_Men_pauv           float64
 19  carreau_Men_1ind           float64
 20  ca

Afin de s'assurer de la correspondance parfaite entre les ventes réalisées par l'agence et celles entrées dans `france.csv` nous effectuons un inner merge sur l'identifiant de la parcelle concernée et sur la date de l'opération.

In [12]:
df_enriched1 = pd.merge(agence, france_HG, how='inner', on=['id_parcelle', 'date_mutation'])
df_enriched1.shape

(4000, 57)

Les colonnes communes sont désormais préfixées, il convient de retirer ces préfixes et de supprimer les colonnes dupliquées

In [13]:
(df_enriched1['nom_commune_x'] == df_enriched1['nom_commune_y']).all()

True

In [14]:
temp_df_enriched1 = DfAnalysis(df_enriched1.T)
temp_df_enriched1.duplicate_check()
df_enriched1 = temp_df_enriched1.df.T

df_enriched1.rename(columns={col: col.rsplit('_', 1)[0] for col in df_enriched1.columns if col.endswith('_x')}, inplace=True)

df_enriched1[['code_postal', 'code_commune']] # considérons code commune qui n'équivaut pas au code_postal
df_enriched1['code_commune'] = df_enriched1['code_commune'].astype(str)

There are 6 rows that are duplicated so we need to drop them
After dropping duplicated rows, there are 51 rows left


<font color="red">ATTENTION</font>: `code_commune` ne correspond pas au code postal de la commune mais à l'identifiant attribué par l'Insee à chaque commune de France


## Etapes 3, 4

Nous devons encore ajouter les variables de géolocalisation des communes (elles remplaceront leurs noms et leur code postal puisqu'elle sont uniques) et nous permettront de calculer la distance au centre ville de la commune de rattachement du bien ainsi que la distance au centre ville de toulouse (deux nouvelles features). 

- `insee_commune.csv` est un dataset essentiel pour notre étude car il contient des informations socio économiques relatives aux communes que l'on considèrera
- `localisation_commune.csv` est moins riche mais non moins intéressant: il contient notamment les coordonnées gps des centres ville des communes étudiées ici 

Encore une fois on sélectionne la plage de données concernant la Haute-Garonne

In [15]:
localisation_commune['Code_postal'] = localisation_commune['Code_postal'].astype(str)
mask = localisation_commune['Code_postal'].str.startswith('31')
localisation_commune_HG = localisation_commune[mask]

In [16]:
localisation_commune_HG['coordonnees_gps'].apply(lambda s: type(s)).unique()

array([<class 'str'>], dtype=object)

Nous séparons la colonne `coordonnees_gps` en deux nouvelles colonnes `latitude_comm` et 'longitude_comm' pour préciser qu'il s'agit des coordonnées du centre ville de la commune

In [17]:
localisation_commune_HG[['latitude_comm', 'longitude_comm']] = localisation_commune_HG['coordonnees_gps'].str.split(',', expand=True)

localisation_commune_HG['latitude_comm'] = pd.to_numeric(localisation_commune_HG['latitude_comm'], errors='coerce')
localisation_commune_HG['longitude_comm'] = pd.to_numeric(localisation_commune_HG['longitude_comm'], errors='coerce')

print(localisation_commune_HG[['latitude_comm', 'longitude_comm']])

       latitude_comm  longitude_comm
269        46.460771        2.559048
270        46.219967        3.249307
273        46.401575        2.730717
281        46.266258        3.229794
286        46.269689        3.423880
...              ...             ...
33986      43.663774        1.677386
33987      43.750880        1.490977
33988      43.406779        1.719651
33989      43.828733        1.496849
33990      43.134325        0.485144

[690 rows x 2 columns]


In [18]:
localisation_commune_HG.rename(columns={'Code_commune_INSEE': 'code_commune'}, inplace=True)

Nous ne nous donnons pas la peine de sélectionner les communes de la Haute-Garonne de `insee_commune.csv`, nous les obtiendrons naturellement lorsque nous fusionneront ce dataset avec `localisation_commune_HG.csv`

In [19]:
insee_commune.rename(columns={'CODGEO':'code_commune'}, inplace=True)

Nous nous séparons des variables parasites

In [20]:
localisation_commune_HG = localisation_commune_HG.drop(['Ligne_5',
       'Libellé_d_acheminement', 'Libell_d_acheminement', 'coordonnees_gps'], axis=1)

Nous fusionnons nos deux datasets sur ce fameux `code_commune`

In [21]:
df_enriched2 = pd.merge(insee_commune, localisation_commune_HG, how='inner', on='code_commune')

Nous vérifions que nos deux datasets ne disposent pas d'autres colonnes en commun que la colonne de fusion afin d'éviter de potentiels problèmes de dimension

In [22]:
df_enriched2.columns[(df_enriched2.columns.isin(df_enriched1.columns))]

Index(['code_commune'], dtype='object')

In [23]:
df_enriched2_unique = df_enriched2.drop_duplicates(subset=['code_commune'], keep='first')
print('df2 dropping duplicate for code_commune: ', df_enriched2_unique.shape)
print('df2: ', df_enriched2.shape)

df2 dropping duplicate for code_commune:  (674, 40)
df2:  (690, 40)


In [24]:
print(df_enriched1['code_commune'].apply(lambda s: type(s)).unique())
print(df_enriched2['code_commune'].apply(lambda s: type(s)).unique())

[<class 'str'>]
[<class 'str'>]


On vérifie que l'on peut bien se permettre de supprimer les index pour lesquels la valeur en `code_commune` est égale, il s'agit en fait que tout l'index est alors dupliqué, on peut donc le supprimer

In [25]:
doublons_mask = df_enriched2.duplicated(subset=['code_commune'], keep=False)
duplicate_rows = df_enriched2.loc[doublons_mask]
print(duplicate_rows.head(2))

   code_commune       LIBGEO  REG DEP  P16_POP  P11_POP  SUPERF  NAIS1116  \
46        03158  Haut-Bocage   84  03      892      965   70.64        33   
47        03158  Haut-Bocage   84  03      892      965   70.64        33   

    DECE1116     P16_MEN  ...  ETFZ15  ETGU15  ETGZ15  ETOQ15  ETTEF115  \
46        29  361.206154  ...     8.0    34.0     5.0    10.0      17.0   
47        29  361.206154  ...     8.0    34.0     5.0    10.0      17.0   

    ETTEFP1015  Nom_commune  Code_postal  latitude_comm  longitude_comm  
46         2.0  HAUT BOCAGE         3190      46.492864        2.658524  
47         2.0  HAUT BOCAGE         3190      46.492864        2.658524  

[2 rows x 40 columns]


Nous obtenons finalement notre dataset final en effectuant un merge left toujours sur le `code_commune` nous avons bien conservé la forme du dataset original, c'est bon signe:

In [26]:
df_final = pd.merge(df_enriched1, df_enriched2_unique, how='left', on='code_commune')

In [27]:
df_final.shape # df_final a une forme correcte, égale à celle d'agence_csv

(4000, 90)

## Etapes 5, 6

Nous devons éliminer quelques-unes des features qui ne sont pas pertinentes ou qui présentent des risques de data leakage 

In [28]:
print(f'Nous avons {df_final.shape[1]} features')

Nous avons 90 features


In [29]:
df_final_red = df_final

In [30]:
# data leakage / index du df de droite
df_final_red.drop(['prixcible', 'prixvente_initial', 'index_right'], axis=1, inplace=True) 

# features non pertinentes
df_final_red.drop(
    
    ['adresse_code_voie', 'code_commune','code_postal', 'nom_commune',
       'id_parcelle','LIBGEO', 'REG', 'DEP', 'Nom_commune','Code_postal', 
       'departement']
       
       , axis=1, inplace=True)

In [31]:
df_final_red.columns

Index(['date_mutation', 'valeur_fonciere', 'type_local', 'surface_reelle_bati',
       'surface_terrain', 'nombre_pieces_principales', 'agent',
       'evaluation_agent', 'code_postal_y', 'longitude', 'latitude',
       'carreau_Id_carr_n', 'carreau_Ind', 'carreau_Men', 'carreau_Men_pauv',
       'carreau_Men_1ind', 'carreau_Men_5ind', 'carreau_Men_prop',
       'carreau_Men_fmp', 'carreau_Ind_snv', 'carreau_Men_surf',
       'carreau_Men_coll', 'carreau_Men_mais', 'carreau_Log_av45',
       'carreau_Log_45_70', 'carreau_Log_70_90', 'carreau_Log_ap90',
       'carreau_Log_inc', 'carreau_Log_soc', 'carreau_Ind_0_3',
       'carreau_Ind_4_5', 'carreau_Ind_6_10', 'carreau_Ind_11_17',
       'carreau_Ind_18_24', 'carreau_Ind_25_39', 'carreau_Ind_40_54',
       'carreau_Ind_55_64', 'carreau_Ind_65_79', 'carreau_Ind_80p',
       'carreau_Ind_inc', 'carreau_I_pauv', 'carreau_t_maille', 'P16_POP',
       'P11_POP', 'SUPERF', 'NAIS1116', 'DECE1116', 'P16_MEN', 'NAISD18',
       'DECESD18', 'P16

Les données statistiques des carreaux sont des données numériques

In [32]:
carreau_to_numeric = [col for col in df_final_red.columns if col.startswith('carreau')]

df_final_red[carreau_to_numeric] = df_final_red[carreau_to_numeric].apply(pd.to_numeric, errors='coerce')

In [33]:
df_final_red['date_mutation'] = pd.to_datetime(df_final_red['date_mutation'], format='%Y-%m-%d')

In [34]:
df_final_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 76 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_mutation              4000 non-null   datetime64[ns]
 1   valeur_fonciere            4000 non-null   object        
 2   type_local                 4000 non-null   object        
 3   surface_reelle_bati        4000 non-null   object        
 4   surface_terrain            4000 non-null   object        
 5   nombre_pieces_principales  4000 non-null   object        
 6   agent                      4000 non-null   object        
 7   evaluation_agent           4000 non-null   object        
 8   code_postal_y              4000 non-null   object        
 9   longitude                  4000 non-null   object        
 10  latitude                   4000 non-null   object        
 11  carreau_Id_carr_n          0 non-null      float64       
 12  carrea

Toutes les valeurs de `carreau_Id_carr_n` sont nulles, on supprime donc cette colonne

In [35]:
df_final_red.drop('carreau_Id_carr_n', axis=1, inplace=True)

Nous convertissons enfin les variables restantes

In [36]:
cols_to_numeric = ['valeur_fonciere', 'surface_reelle_bati',
                    'surface_terrain', 'nombre_pieces_principales',
                    'longitude', 'latitude']

df_final_red[cols_to_numeric] = df_final[cols_to_numeric].apply(pd.to_numeric, errors='coerce')

In [37]:
df_final_red.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 75 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   date_mutation              4000 non-null   datetime64[ns]
 1   valeur_fonciere            4000 non-null   float64       
 2   type_local                 4000 non-null   object        
 3   surface_reelle_bati        4000 non-null   float64       
 4   surface_terrain            4000 non-null   float64       
 5   nombre_pieces_principales  4000 non-null   float64       
 6   agent                      4000 non-null   object        
 7   evaluation_agent           4000 non-null   object        
 8   code_postal_y              4000 non-null   object        
 9   longitude                  4000 non-null   float64       
 10  latitude                   4000 non-null   float64       
 11  carreau_Ind                4000 non-null   float64       
 12  carrea

**Sélection de variable**

Nous commençons tout d'abord par générer la matrice des corrélations:
- (voir dossier img $\rightarrow$ correlation_matrix.png)
- Nous remarquons que de très nombreuses variables sont très fortement corrélées, il y a donc matière à sélection

In [38]:
dan_df_final_red = DfAnalysis(df_final_red)
dan_df_final_red.correlation_matrix() # beaucoup de variables sont fortement corrélées

Avant de sélectionner nos variables nous devons nous occuper des index pour lesquels des données manquent, nous pourrions: supprimer les index, remplir artificiellement les données manquantes avec filna s'il s'agit de données numériques ou supprimer les features concernées si celles-ci ne s'avèrent pas être d'un grand intérêt pour notre étude.

In [39]:
pd.set_option('display.max_columns', None)
cols_with_nan = df_final_red.loc[3786].isna()

# Afficher les noms des colonnes qui ont une valeur NaN pour cet index
print(cols_with_nan[cols_with_nan].index.tolist())

df_final_clean = df_final_red

df_final_clean.drop(['PIMP16', 'TP6016'], axis=1, inplace=True)

['PIMP16', 'TP6016']


`PIMP16`
Part des ménages fiscaux imposés (%)

`TP6016`
Taux de pauvreté-Ensemble (%)

On peut lire sur la matrice de corrélation que ces deux variables sont toutes deux très corrélées à au moins une autre variable (au delà de 75% en valeur absolue) on peut donc raisonnablement se permettre de les supprimer

In [40]:
chemin_dossier = r'C:\Users\User\Desktop\Projets en cours\Data_Science\Mini-projets\Case_cap_gemini_emmanuel_benichou\df_ml'
nom_fichier = 'df_for_ml.csv'
chemin_complet = os.path.join(chemin_dossier, nom_fichier)

os.makedirs(chemin_dossier, exist_ok=True)

df_final_clean.to_csv(chemin_complet, index=False, encoding='utf-8')

**Feature engineering**

Nous ajoutons à présent les distances que nous avions mentionnées tout à l'heure

ATTENTION le df est largement déséquilibré (voir le nombre de ventes à toulouse représentée ici par sa longitude et latitude)

## Etapes 7, 8

In [41]:
cat_cols = [col for col in df_final_clean.select_dtypes(include='object').columns]
df_final_clean_ml = DataPreprocessing(df_final_clean, columns_to_exclude=['date_mutation'], categorical_features=cat_cols)
df_final_clean_ml.scale_selected_columns()
X_train, X_test, y_train, y_test = df_final_clean_ml.split_data('valeur_fonciere')
ml_df = df_final_clean_ml.scaled_df

La PCA ne semble pas être concluante, elle combine de trop nombreuses variables corrélées pour avoir du sens

**Implémentation de notre premier modèle de machine learning**

L'objectif ici est de tester notre dataset, d'essayer d'évaluer sans sélection préalable de variable le niveau d'overfitting d'un modèle performant pour la régression, nous utilisons catboost regressor, un algorithme de forêts d'arbres de décisions avec gradient boosting prenant plutôt bien en charge les hautes dimensionnalités ainsi que les variables catégorielles (au nombre de 4 dans notre dataset)

In [42]:
cat = CatBoostRegressor(loss_function='RMSE', cat_features=cat_cols, verbose=False)

param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 500]
}

grid_search = GridSearchCV(estimator=cat, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)

grid_search.fit(X_train, y_train)
best_cat = grid_search.best_estimator_
y_pred = best_cat.predict(X_test)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


In [43]:
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print(100 * '-')
print("R2 score:", r2_score(y_test, y_pred))
print(100 * '-')
print('Best CatBoost parameters:', grid_search.best_params_)

Mean Squared Error: 0.15588928477376396
----------------------------------------------------------------------------------------------------
R2 score: 0.8463892029430315
----------------------------------------------------------------------------------------------------
Best CatBoost parameters: {'depth': 4, 'iterations': 500, 'learning_rate': 0.05}


Nous atteignons une profondeur de 4, 500 apprenants faibles, un learning rate presque minimal ainsi qu'une excellente performance en terme de R2 score: tout ceci suggère très fortement de l'overfitting, il va donc falloir réduire notre dimensionnalité tout en veillant à préserver une certaine explicabilité de notre modèle. Nous testerons également plusieurs modèles en pipeline sur des jeux de features sélectionnées ou non afin d'établir une comparaison.

In [44]:
feature_importance = best_cat.feature_importances_

importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importance:")
print(importance_df)

Feature Importance:
                Feature  Importance
0   surface_reelle_bati   57.595213
1       surface_terrain   12.995068
69     evaluation_agent    6.235640
4              latitude    3.258643
3             longitude    2.824106
..                  ...         ...
41              NAISD18    0.000000
20      carreau_Log_inc    0.000000
38             NAIS1116    0.000000
39             DECE1116    0.000000
35              P16_POP    0.000000

[71 rows x 2 columns]


<font color="red">Note</font>:

Néanmoins le modèle suggère ici une très grande importance de certaines variables, ce qui ne veut pas non plus dire grand chose du fait que de très nombreuses variables sont fortement corrélées ce qui peut nous conduire à une importance accrue de certaines variables au détriment d'autres (leur importance vient 'nourrir' artificiellement celle d'une seule, sélectionnée arbitrairement par le modèle).

Ce n'est donc pas nécessairement une bonne manière de sélectionner des variables.